In [1]:
!pip install -q transformers timm torch torchvision pandas pillow tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')
image_dir = "/content/drive/MyDrive/lora/ghibli-dataset"

Mounted at /content/drive


In [5]:
import torch
from PIL import Image
import pandas as pd
from tqdm import tqdm
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import os

# Load model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "Salesforce/blip2-flan-t5-xl"

processor = Blip2Processor.from_pretrained(model_id)
model = Blip2ForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
model.to(device)

# Preprocessing function (based on filename)
def preprocess_filename(fname):
    name = os.path.splitext(fname)[0]
    return name.replace("_", " ").replace("-", " ").replace("%20", " ")

def caption_image(image_path):
    image = Image.open(image_path).convert("RGB")
    prompt = "Describe this Ghibli style image."
    inputs = processor(images=image, text=prompt, return_tensors="pt").to(
        device, torch.float16 if torch.cuda.is_available() else torch.float32
    )
    out = model.generate(**inputs, max_new_tokens=40)
    return processor.decode(out[0], skip_special_tokens=True).strip()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
results = []

print(f"📸 Generating captions for {len(os.listdir(image_dir))} images...\n")
for fname in tqdm(os.listdir(image_dir)):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
        continue
    try:
        caption = caption_image(os.path.join(image_dir, fname))
        print(f"🖼️ {fname} → 📝 {caption}")
        results.append({"file_name": fname, "caption": caption})
    except Exception as e:
        print(f"❌ Error with {fname}: {e}")

# Save to CSV
df = pd.DataFrame(results)
df.to_csv(os.path.join(image_dir, "metadata.csv"), index=False)
print("✅ Captions saved to metadata.csv")

📸 Generating captions for 626 images...



  0%|          | 1/626 [00:00<06:31,  1.60it/s]

🖼️ Lx4OpET.jpeg → 📝 a forest scene with trees and flowers


  0%|          | 3/626 [00:01<04:27,  2.33it/s]

🖼️ studio-ghibli-desktop-sky-forest-ona981pslzx33lcp.jpg → 📝 a sailor in a forest


...

🖼️ connor-sheehan-jinji-grotto-enviro-001.jpg → 📝 a troll, a fox, a fox, a fox, a fox, a fox, a fox, a fox


100%|█████████▉| 625/626 [12:04<00:01,  1.17s/it]

🖼️ oday-abuzaeed-oday-abuzaeed-cover-project.jpg → 📝 a sandbox with a wooden slat


100%|██████████| 626/626 [12:04<00:00,  1.16s/it]

🖼️ tyler-edlin-forest-color522.jpg → 📝 a girl is standing in front of a giant sphere
✅ Captions saved to metadata.csv


In [8]:
import pandas as pd

df = pd.read_csv(r"/content/drive/MyDrive/lora/ghibli-dataset/metadata.csv")
df.head(10)

,file_name,caption
0,Lx4OpET.jpeg,a forest scene with trees and flowers
1,studio-ghibli-desktop-sky-forest-ona981pslzx33...,a sailor in a forest
2,gg9CoaM.jpeg,a forest path with a tree and a tree
3,tpOAF3p.jpeg,a forest path in the woods
4,couple.jpg,a girl and boy are reading a book together
5,kimitachi001.jpg,a girl in a kimono is standing in front of a hut
6,kimitachi002.jpg,a group of old ladies with a cup of tea
7,kimitachi027.jpg,a stork with a hat
8,kimitachi003.jpg,a boy in a blue shirt and a blue shirt is stan...
9,kimitachi028.jpg,a boy with a hat and a hat
